In [31]:
import numpy as np 
import pandas as pd

In [32]:
#loading the data
workshop_data = pd.read_csv("C:/Users/abhin/Downloads/workshops_dataset.csv")
students_data = pd.read_csv("C:/Users/abhin/Downloads/student_data.csv")

In [33]:
workshop_data.head()

,Title,Club,Tags
0,CONVenient,"ACM, Manipal","Neural Networks, Convolution Operation, Convol..."
1,Fusion 360 Generative Design Workshop,DronAid,"Fusion 360, Sketching, 3D Modelling, Assemblie..."
2,PCB Designing,DronAid,"PCB Design, KiCad, Electronic Components, Buck..."
3,Astravaganza,"The Astronomy Club, Manipal","Stargazing, Astrophotography, Telescopes, Sequ..."
4,App Development & Backend Workshop,Apple Developers Group,"React Native, TypeScript, Node.js, Mongoose, E..."


In [34]:
students_data.head()

,Name,Reg_No,Branch,Interests
0,Student001,REG0001,Mathematics & Computing,"Probability Models, Data Science, Algorithm De..."
1,Student002,REG0002,Mathematics & Computing,"Algorithm Design, Probability Models, Cryptogr..."
2,Student003,REG0003,Information Technology,"Web Development, Database Management, Cloud Co..."
3,Student004,REG0004,Automobile Engineering,"Automotive Design, EV Technologies, Vehicle Dy..."
4,Student005,REG0005,Aeronautical Engineering,"Aerospace Materials, Propulsion Systems, Space..."


In [35]:
#concatinating branch and interests fields
students_data['Branch_Interests'] = students_data['Branch'] + ', ' + students_data['Interests']

In [36]:
students_data.head()

,Name,Reg_No,Branch,Interests,Branch_Interests
0,Student001,REG0001,Mathematics & Computing,"Probability Models, Data Science, Algorithm De...","Mathematics & Computing, Probability Models, D..."
1,Student002,REG0002,Mathematics & Computing,"Algorithm Design, Probability Models, Cryptogr...","Mathematics & Computing, Algorithm Design, Pro..."
2,Student003,REG0003,Information Technology,"Web Development, Database Management, Cloud Co...","Information Technology, Web Development, Datab..."
3,Student004,REG0004,Automobile Engineering,"Automotive Design, EV Technologies, Vehicle Dy...","Automobile Engineering, Automotive Design, EV ..."
4,Student005,REG0005,Aeronautical Engineering,"Aerospace Materials, Propulsion Systems, Space...","Aeronautical Engineering, Aerospace Materials,..."


In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#creating a TfidfVectorizer object
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(workshop_data['Tags'])

#transforming user data
user_tfidf_matrix = tfidf.transform(students_data['Branch_Interests'])

#calculating cosine similarity
cosine_sim = cosine_similarity(user_tfidf_matrix, tfidf_matrix)

In [38]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

# Apply stemming to workshop tags and student interests to standardize word forms
# Stemming reduces words to their root form (e.g., "programming" -> "program"),
# ensuring uniformity across the tags and interests. This helps improve matching
# between workshops and student preferences by handling variations in phrasing
# (e.g., "computing" vs. "compute").

workshop_data['Tags'] = workshop_data['Tags'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))
students_data['Branch_Interests'] = students_data['Branch_Interests'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

In [39]:
# For each student, get the top-k workshops (e.g., k=3)
top_k = 3
recommendations = []

for student_idx in range(cosine_sim.shape[0]):
    # Get sorted indices of workshops based on similarity
    similar_workshops = cosine_sim[student_idx].argsort()[::-1][:top_k]
    recommendations.append(workshop_data.iloc[similar_workshops]['Title'].values)

In [40]:
#code for popular workshops

In [41]:
students_data['Recommended_Workshops'] = recommendations

In [42]:
students_data.to_csv('student_recommendations.csv', index=False)

In [43]:
cosine_sim

array([[0.        , 0.33817743, 0.21556339, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.28673546,
        0.        , 0.        ],
       [0.        , 0.26942171, 0.17173664, 0.        , 0.        ,
        0.18223091, 0.        , 0.        , 0.        , 0.22843853,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.15075567, 0.        , 0.        , 0.        , 0.37796447,
        0.15075567, 0.        ],
       [0.        , 0.51905848, 0.33086183, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.4485729 , 0.28593243, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.15170241,

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import ast

# Load existing user data
users = pd.read_csv("C:/Users/abhin/OneDrive/Documents/Recommendation-system/student_recommendations.csv")

# Fix the 'Recommended_Workshops' column: Convert string representation to lists
users['Recommended_Workshops'] = users['Recommended_Workshops'].apply(ast.literal_eval)

# Preprocess the new user's data
#this preprocessing may change wrt to the data
def preprocess_user(new_user):
    new_user['Branch_Interests'] = new_user['Branch'] + ', ' + new_user['Interests']
    if 'Recommended_Workshops' not in new_user:
        new_user['Recommended_Workshops'] = []
    return new_user

# Function to recommend workshops
def recommend_workshops(new_user, users, top_n=2):
    # Preprocess new user
    new_user = preprocess_user(new_user)
    
    # Combine new user data with existing users for TF-IDF vectorization
    combined_data = users['Branch_Interests'].tolist() + [new_user['Branch_Interests']]
    
    # Create TF-IDF matrix
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(combined_data)
    
    # Calculate cosine similarity
    cosine_sim = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1]).flatten()
    
    # Find top N similar users
    similar_user_indices = cosine_sim.argsort()[-top_n:][::-1]
    
    # Collect workshops from similar users
    recommended_workshops = set()
    for idx in similar_user_indices:
        recommended_workshops.update(users.iloc[idx]['Recommended_Workshops'])
    
    # Exclude workshops already attended by the new user
    recommended_workshops.difference_update(new_user['Recommended_Workshops'])
    

# Example new user
new_user = {
    "Name": "Ram",
    "Reg_No": "Student069",
    "Branch": "Computer Science",
    "Interests": "Artificial Intelligence, Data Science, Web Development",
}

# Get recommendations
recommendations = recommend_workshops(new_user, users)
print("Recommended Workshops:", recommendations)

Recommended Workshops: ['Azure Pathways: Intro To Cloud Computing with AzureHacklinux: Cryptography and Web Exploitation WorkshopVR Arcade Basketball: Game Development Workshop', 'Fusion 360 Generative Design WorkshopAzure Pathways: Intro To Cloud Computing with AzurePCB Designing']
